<a href="https://colab.research.google.com/github/levietduc1007/machine-learning-regression-exercises/blob/main/PINNs_Framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Framework cho tất cả các dạng toán
Ký hiệu:
*   x: input
*   u: output




In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
from pyDOE import lhs
import time
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
import os

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Đề: Linear regression

In [ ]:
x_train = [[1.0], [2.0], [3.0], [4.0], [5.0]]
y_train = [[3.0], [5.0], [7.0], [9.0], [11.0]]

# Xây dựng Neural

In [ ]:
class PhysicsInformedNN(nn.Module):
  # ép dữ liệu về kiểu pytorch
    def __init__(self, input_size, output_size, hidden_size=[20, 20, 5], activation='tanh'):
        super(PhysicsInformedNN, self).__init__()
        layers_list = []

        # Nhận dạng hàm activation
        for i in range(len(hidden_size)):
            layers_list.append(nn.Linear(self.layers_list[i], self.layers_list[i+1])) #input layer i là output layer i+1

            # Không thêm hàm kích hoạt sau lớp cuối cùng
            if i < len(hidden_size) - 1:
                if activation == 'relu':
                    layers_list.append(nn.ReLU())
                elif activation == 'sigmoid':
                    layers_list.append(nn.Sigmoid())
                elif activation == 'tanh':
                    layers_list.append(nn.Tanh())
                else:
                    layers.append(nn.Softplus())

        # Khởi tạo Xavier (glorot_normal) cho các lớp Linear
        for layer in layers_list:
            if isinstance(layer, nn.Linear): # chỉ lọc hàm linear
                nn.init.xavier_normal_(layer.weight)
                nn.init.zeros_(layer.bias)

        return nn.Sequential(*layers_list) # Không hiểu cái này là gì, đọc rồi vẫn không hiểu, chỉ biết là cần tại chỗ này


    def forward(self, X):
        H = 2.0 * (X - self.lb) / (self.ub - self.lb) - 1.0 # Chuẩn hóa input để từ -1 đến 1, để tránh ảnh hưởng bởi tham số lớn
        return self.net(H) # self.net chứa NN đã được tạo từ nn.Sequential

##################### Dự đoán u v và đạo hàm output theo input ###########################################

    def get_uv_and_derivatives(self, x):
        # Báo cho PyTorch theo dõi đạo hàm của input
        x.requires_grad_(True)

        x = torch.cat([x, t], dim=1) # gộp lại thành matrix input duy nhất
        u_pred = self(x) # chạy NN ra dự đoán u_pred

        # Tách rời thành từng tensor output
        u = uv[:, 0:1] # gần giống uv[:, 0], khác ở chỗ uv[:, 0:1] lấy cột ra nhưng vẫn giữ nguyên vỏ bọc 2 chiều.
        # Ex: [[0.5], [0.8], [0.1]] thay vì [0.5, 0.8, 0.1]

        # Yêu cầu để tính đạo hàm cho đầu ra đa chiều
        grad_outputs = torch.ones_like(u, device=device) # tạo matrix kích thước giống u với full 1

        # Tính đạo hàm bậc 1
        # create_graph=True là cực kỳ quan trọng, nó cho phép tính đạo hàm bậc 2
        #u_t = torch.autograd.grad(u, t, grad_outputs=grad_outputs, create_graph=True)[0]
        #v_t = torch.autograd.grad(v, t, grad_outputs=grad_outputs, create_graph=True)[0]
        u_x = torch.autograd.grad(u, x, grad_outputs=grad_outputs, create_graph=True)[0]
        #v_x = torch.autograd.grad(v, x, grad_outputs=grad_outputs, create_graph=True)[0]

        return u, u_x

####################### Tính toán residual của PDE (f_u, f_v)##################################

    def get_f_uv(self, x, t):

        u, u_x = self.get_uv_and_derivatives(x)

        grad_outputs = torch.ones_like(u_x, device=device) # tương tự, hỏi thầy ?????

        # Tính đạo hàm bậc 2
        u_xx = torch.autograd.grad(u_x, x, grad_outputs=grad_outputs, create_graph=True)[0]
        #v_xx = torch.autograd.grad(v_x, x, grad_outputs=grad_outputs, create_graph=True)[0]

        # Tính f_u, f_v theo công thức PDE, ở đây có 2 pt residual theo 2 biến thì có 2 phần
        # f_u =
        # f_v =

        return f_u, f_v

############### Tính toán tổng loss ###############
    def compute_loss(self):

        # 1. Loss từ PDE residual (f)
        f_u_pred, f_v_pred = self.get_f_uv(self.x_f, self.t_f) # tính residual
        loss_f = torch.mean(f_u_pred**2) + torch.mean(f_v_pred**2) # Hàm Loss dạng MSE

        # 2. Loss từ điều kiện ban đầu (t=0)
        u0_pred, v0_pred, _, _, _, _ = self.get_uv_and_derivatives(self.x0, self.t0)
        loss_0 = torch.mean((self.u0 - u0_pred)**2) + torch.mean((self.v0 - v0_pred)**2)

        # 3. Loss từ điều kiện biên (lb, ub)
        x_lb_with_grad = self.x_lb.clone().requires_grad_(True) # Báo cho PyTorch theo dõi đạo hàm của x_lb và x_ub
        x_ub_with_grad = self.x_ub.clone().requires_grad_(True)

        u_lb, v_lb, _, _, u_x_lb, v_x_lb = self.get_uv_and_derivatives(x_lb_with_grad, self.t_b)
        u_ub, v_ub, _, _, u_x_ub, v_x_ub = self.get_uv_and_derivatives(x_ub_with_grad, self.t_b)

        loss_b = torch.mean((u_lb - u_ub)**2) + \
                 torch.mean((v_lb - v_ub)**2) + \
                 torch.mean((u_x_lb - u_x_ub)**2) + \
                 torch.mean((v_x_lb - v_x_ub)**2)

        # Tổng loss
        return loss_0 + loss_b + loss_f

############### Vòng huấn luyện chính ###############
    def train_model(self, nIter):
        # Dùng Adam optimizer
        optimizer = torch.optim.Adam(self.parameters())

        start_time = time.time() # bắt đầu set time hiện tại là 0
        for it in range(nIter):
            # BƯỚC 1: Xóa đạo hàm cũ
            optimizer.zero_grad()

            # BƯỚC 2: Tính loss
            loss = self.compute_loss()

            # BƯỚC 3: Tính đạo hàm của loss theo các trọng số (backpropagation)
            loss.backward()

            # BƯỚC 4: Cập nhật trọng số
            optimizer.step()
            # các tham số (weights, bias) của mô hình đã được cập nhật theo đúng quy tắc của Adam Optimizer.

            # In ra thông tin
            if it % 10 == 0:
                elapsed = time.time() - start_time
                print(f'It: {it}, Loss: {loss.item():.3e}, Time: {elapsed:.2f}')
                start_time = time.time()

    def predict(self, X_star):
        # Dự đoán
        # Chuyển numpy array sang tensor
        X_star_torch = torch.tensor(X_star, dtype=torch.float32).to(device)
        x_tf = X_star_torch[:, 0:1].clone().requires_grad_(True)
        t_tf = X_star_torch[:, 1:2].clone().requires_grad_(True)

        # Chuyển sang chế độ đánh giá (tắt dropout, v.v.)
        self.eval()

        # Lấy dự đoán u, v
        uv_star_torch = self(torch.cat([x_tf, t_tf], dim=1))
        u_star = uv_star_torch[:, 0:1]
        v_star = uv_star_torch[:, 1:2]

        # Lấy dự đoán f_u, f_v
        f_u_star, f_v_star = self.get_f_uv(x_tf, t_tf)

        # Chuyển về chế độ huấn luyện
        self.train()

        # Chuyển kết quả về lại numpy (chuyển về CPU, tách khỏi đồ thị, rồi convert)
        return u_star.cpu().detach().numpy(), \
               v_star.cpu().detach().numpy(), \
               f_u_star.cpu().detach().numpy(), \
               f_v_star.cpu().detach().numpy()


In [ ]:
# 4. Huấn luyện mô hình
epochs = 200
for epoch in range(epochs):
    # Forward
    y_pred = model(x_train)
    loss = criterion(y_pred, y_train)

    # Backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 20 == 0:
        print(f'Epoch {epoch+1}: loss={loss.item():.4f}')

# 5. Dự đoán với dữ liệu mới
x_test = torch.tensor([[6.0]])
y_test = model(x_test)
print("Dự đoán y khi x=6:", y_test.item())